In [1]:
# packages
import os
import numpy as np
import pandas as pd
import pickle
from scipy import interpolate
import matplotlib.pyplot as plt
import matplotlib as mpl
from multiprocessing import Pool
from source.model_2state import solve_prep, solve_pre_jump_2state
import plotly.graph_objects as go
from plotly.subplots import make_subplots
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams["figure.edgecolor"] = "w"
mpl.rcParams['figure.facecolor'] = "w"
mpl.rcParams["savefig.dpi"] = 300

# Appendix D: when $Y$ has two states:

Process:

$$
dY_t^1  = Y_t^2 dt
$$

$$
dY_t^2 =   - \lambda Y_t^2 dt + \lambda \theta \mathcal{E} dt
$$


## Step I: post jump HJB:

The post jump HJB for $\phi_m(y_1, y_2)$, $m = 1, \dots, M$:

$$
\begin{aligned}
0 = \max_{\mathcal{E}} \min_{\omega_\ell } & - \delta \phi_m(y_1, y_2) + \eta log(\mathcal{E}) \\
& + \frac{\partial \phi_m}{\partial y_1} y_2 + \frac{\partial \phi_m}{\partial y_2} \lambda (- y_2 + \sum_{\ell = 1}^L 
\omega_\ell  \theta_\ell \mathcal{E}) \\ 
& + \frac{(\eta - 1)}{\delta} \left(\gamma_1 + \gamma_2 y_1 + \gamma_3 (y_1 - \bar y)\mathbb{I}\{y_1>\bar y\} \right) y_2 \\
& + \xi_a \sum_{\ell = 1}^L \omega_\ell (\log \omega_\ell - \log \pi^a_\ell)
\end{aligned}
$$

First order condition for $\omega_\ell$, $\ell = 1, \dots, L$:

$$
    \omega_\ell \propto \pi_\ell^a \exp\left( -\frac{1}{\xi_a} \frac{\partial \phi_m}{\partial y_2}\lambda \theta_\ell \mathcal{E} \right), for \ell = 1, \dots, L
$$

and the first order condition for emission is:
$$
\mathcal{E} = - \cfrac{\eta}{\frac{\partial \phi_m }{\partial y_2} \lambda \sum_{\ell=1}^{L} \omega_\ell \theta_\ell}
$$


## Step II: pre jump HJB:

Given post jump value functions $\phi_m(y_1, y_2)$, $m = 1, 2, M$, solve the following HJB for pre-jump value function $\Phi(y_1, y_2)$:

$$
\begin{aligned}
0 = \max_{\mathcal{E}}\min_{\omega_\ell }  & - \delta \Phi(y_1, y_2) +  \eta log(\mathcal{E}) \\
& + \frac{\partial \Phi}{\partial y_1} y_2 + \frac{\partial \Phi}{\partial y_2} \lambda (- y_2+ \sum_{\ell = 1}^L 
\omega_\ell  \theta_\ell \mathcal{E}) \\ 
& + \frac{(\eta - 1)}{\delta} (\gamma_1 + \gamma_2 y_1 ) y_2 \\
& + \xi_a \sum_{\ell = 1}^L \omega_\ell (\log \omega_\ell - \log \pi^a_\ell)\\
& + \mathcal{J}(y_1) \sum_{m=1}^M g_m \pi_d^m ( \phi_m(\bar{y}_1, y_2) - \Phi(y_1, y_2)) \\
& + \xi_p \mathcal{J}(y_1) \sum_{m=1}^M \pi_d^m \left(1 - g_m + g_m \log (g_m)\right)
\end{aligned}
$$

Or solve the following HJB with a terminal condition:

$$
\begin{aligned}
0 = \max_{\mathcal{E}}\min_{\omega_\ell }  & - \delta \Phi(y_1, y_2) +  \eta log(\mathcal{E}) \\
& + \frac{\partial \Phi}{\partial y_1} y_2 + \frac{\partial \Phi}{\partial y_2} \lambda (- y_2+ \sum_{\ell = 1}^L 
\omega_\ell  \theta_\ell \mathcal{E}) \\ 
& + \frac{(\eta - 1)}{\delta} (\gamma_1 + \gamma_2 y_1 ) y_2 \\
& + \xi_a \sum_{\ell = 1}^L \omega_\ell (\log \omega_\ell - \log \pi^a_\ell)
\end{aligned}
$$

$$
\Phi(\bar y_1, y_2) \approx  - \xi_p \log \left (\sum_{m=1}^M \pi_m^p \exp\left[-\frac{1}{\xi_p }\phi_m(\bar y_1, y_2) \right] \right) 
$$

In what follows, we show emission, $Y$ behavior before temperature anomay reaches the upper bound of jump threshold, $2^o C$, conditioning on no jump.

The $\lambda$ we choose here are:
- $\lambda = 0.116$, corresponds to half life of six years;
- $\lambda = 1, 2, 5$. Here, we try to illustrate emission behavior as $\lambda$ increase.

## Example 1: Half life of six years

In [ ]:
!cd src && python remark3p1.py -L 0.116

In [2]:
λ = np.log(2) / 6
# parameters
δ = 0.01
η = 0.032
ξa = 0.01
θ_list = pd.read_csv("./data/model144.csv", header=None)[0].to_numpy()
θ_list = θ_list/1000
θ = np.mean(θ_list)
σy = 1.2*θ
# damage function
y_bar = 2.
γ1 = 0.00017675
γ2 = 2*0.0022
γ3_list = np.linspace(0., 1./3, 20)
# %%
# y_grid
y1_step = .04
y1_grid = np.arange(0., 4., y1_step)

y2_step = .001
y2_grid = np.arange(0., .05, y2_step)

args_list  = []
for γ3_i in γ3_list:
    args_iter = (y1_grid, y2_grid, γ3_i, θ_list, (δ, η, γ1, γ2, y_bar, λ, ξa), 1e-6, 1., 1000, 0.05)
    args_list.append(args_iter)

if not os.path.exists(f"./data/res_list_{λ}"):
    with Pool() as p:
        res_list = p.starmap(solve_prep, args_list)

    with open(f"./data/res_list_{λ}", "wb") as handle:
        pickle.dump(res_list, handle)

# step II HJB:
with open(f"./data/res_list_{λ}", "rb") as file:
    res_list = pickle.load(file)

ξp = 1.
if not os.path.exists(f"./data/res_{λ}_{ξp}"):
    args_pre_jump = (δ, η, θ_list,  γ1, γ2, γ3_list, ξa, ξp)
    res = solve_pre_jump_2state(res_list, args_pre_jump, ε=0.1)
    with open(f"./data/res_{λ}_{ξp}", "wb") as handle:
        pickle.dump(res, handle)
# load results
res = pickle.load(open(f"./data/res_{λ}_{ξp}", "rb"))

In [3]:
def simulation(res, θ=1.86/1000., y1_0=1.1, y2_0=1.86/1000, T=100):
    y1_grid = res["y1"]
    y2_grid = res["y2"]
    e_grid = res["ems"]
    λ = res["λ"]
    e_fun = interpolate.interp2d(y1_grid, y2_grid, e_grid.T)
    Et = np.zeros(T+1)
    y1t = np.zeros(T+1)
    y2t = np.zeros(T+1)
    for i in range(T+1):
#         y2_0 = max(y2_0, 0)
#         y2_0 = min(y2_0, 0.05)
        Et[i] = e_fun(y1_0, y2_0)
        y1t[i] = y1_0
        y2t[i] = y2_0
        y2_0 = np.exp(-λ)*y2_0 + (1 - np.exp(-λ))*θ*Et[i] 
#         y2_0 = max(y2_0, 0)
        y1_0 = y1_0 + y2_0
    return Et, y1t, y2t

In [4]:
et_prejump, y1t_prejump, y2t_prejump = simulation(res,
                                                  θ=np.mean(θ_list),
                                                  y1_0 = 1.1,
                                                  y2_0=np.mean(θ_list),
                                                  T=110
                                                 )

In [5]:
simul = {
    "et": et_prejump,
    "y1t": y1t_prejump,
    "y2t": y2t_prejump,
}

pickle.dump(simul, open(f"data/simul_{λ}", "wb"))

In [15]:
fig = make_subplots(cols=3, rows=1)
fig.add_trace(go.Scatter(x=np.arange(len(simul["et"])), y=simul['et'], name="Emission"), col=1, row=1)
fig.add_trace(go.Scatter(x=np.arange(len(simul["et"])), y=simul['y1t'], name="$Y_t^1$"), col=2, row=1)
fig.add_trace(go.Scatter(x=np.arange(len(simul["et"])), y=simul['y2t'], name="$Y_t^2$"), col=3, row=1)
fig.update_layout(
#     legend
    height=300,
    width=1000
)
fig.update_xaxes(title="Years")

Compare the emission trajectories with the one state model:

In [21]:
et_1state = np.load("notebook/et_1state_cal.npy")
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(et_1state)), y=et_1state, name="Y has one state"))
fig.add_trace(go.Scatter(x=np.arange(len(simul["et"])), y=simul["et"], name="Y has two states"))
fig.update_yaxes(range=[0,13])
fig.update_xaxes(title="Years")
fig.update_layout(title="Emissions")

In [22]:
# simulate
def simulate_logkapital(invkap, αₖ,  σₖ, κ, k0, T=100, dt=1):
    periods = int(T/dt)
    Kt = np.zeros(periods)
    i = invkap
     # log capital
    Kt[0] = np.log(k0)
    k = np.log(k0)
    for t in range(periods-1):
        k +=   (αₖ + i - κ/2*i**2 - .5*σₖ**2)*dt
        Kt[t+1] = k
    return Kt

In [23]:
invkap = 0.09
α = 0.115
αₖ = - 0.043
σₖ = 0.0095
κ = 6.667
k0 = 85/α

In [26]:
Kt = simulate_logkapital(invkap, αₖ, σₖ, κ,  k0, T=111)
MC = δ*(1-η)/((α - invkap)*np.exp(Kt))
scc = η*(α - invkap)*np.exp(Kt)/(1-η)/et_prejump*1000
scc_1 = η*(α - invkap)*np.exp(Kt[:len(et_1state)])/(1-η)/et_1state*1000

In [29]:
# TODO: scc comparison
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(scc)), y=scc, name="Y has two states"))
fig.add_trace(go.Scatter(x=np.arange(len(scc_1)), y=scc_1, name="Y has one state"))
fig.update_xaxes(title="Years")
fig.update_yaxes(title="Social cost of carbon")

# Example 2: increasing $\lambda$

For the purpose of illustration, we choose $\lambda = 1, 2, 5$ to show what happens if the half life decreases.

In [ ]:
res_dict = {}
for λ in [1., 2., 5.]:
    args_list  = []
    for γ3_i in γ3_list:
        args_iter = (y1_grid, y2_grid, γ3_i, θ_list, (δ, η, γ1, γ2, y_bar, λ, ξa), 1e-6, 1., 1000, 0.05)
        args_list.append(args_iter)

    if not os.path.exists(f"./data/res_list_{λ}"):
        with Pool() as p:
            res_list = p.starmap(solve_prep, args_list)

        with open(f"./data/res_list_{λ}", "wb") as handle:
            pickle.dump(res_list, handle)

    # step II HJB:
    with open(f"./data/res_list_{λ}", "rb") as file:
        res_list = pickle.load(file)

    args_pre_jump = (δ, η, θ_list,  γ1, γ2, γ3_list, ξa, ξp)
    res = solve_pre_jump_2state(res_list, args_pre_jump, ε=0.1)

    with open(f"./data/res_{λ}_{ξp}", "wb") as handle:
        pickle.dump(res, handle)
    res_dict[λ] = res

In [ ]:
# plot, emission - y, λ = 1,2,5

In [ ]:
# plot, emission, y1, y2, button, λ = 1, 2, 5

In [ ]:
#  emission, λ = 1,2,5 and 1 state result

In [ ]:
# scc λ = 1, 2, 5 and 1 state